# Factor risk models

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx

from cvx.risk.factor.linalg.pca import pca as principal_components
from cvx.risk.factor.timeseries import TimeseriesFactorRiskModel
from cvx.risk.factor.fundamental import FundamentalFactorRiskModel

from cvx.risk.sample.sample import SampleCovariance, SampleCovariance_Product

## An abstract Risk Model

An abstract risk model is the parent for concrete risk models. It serves as the blueprint. No instances of this class can be created.
Each risk model inherits the interface and all attributes defined in this class. 

An optimizer would get a risk model. Since all risk models share the interface it is trivial to change the risk model.

In [2]:
from abc import abstractmethod, ABC


class RiskModel(ABC):
    @abstractmethod
    def estimate_risk(self, weights, assets=None, **kwargs):
        """
        Estimate the variance of a portfolio given its weights
        The weights may come as a cvxpy Variable or numpy array.
        """

## Load prices and compute returns

In [3]:
prices = pd.read_csv("data/stock_prices.csv", index_col=0, header=0, parse_dates=True)
returns = prices.pct_change().fillna(0.0)

# compute vola
# compute ret/vola
# winsorize

# 

## Compute principal components

In [4]:
components = principal_components(returns=returns, n_components=15)

## Create the risk model, here a FactorModel

In [5]:
model = TimeseriesFactorRiskModel(returns=returns, factors=components.returns)
# test the risk model with uniform weights
weights = pd.Series(index = prices.columns, data=0.05).values
model.estimate_risk(weights).value

8.410457095029708e-05

## But we could also use the sample covariance matrix here

In [6]:
#print(returns.cov().shape)
model2 = SampleCovariance(num=20)
model2.cov.value = returns.cov().values

weights = pd.Series(index = prices.columns, data=0.05).values
model2.estimate_risk(weights).value

model2 = SampleCovariance_Product(num=20)
model2.cov.value = returns.cov().values
weights = pd.Series(index = prices.columns, data=0.05).values
model2.estimate_risk(weights).value


array(8.36104368e-05)

## Fundamental Factor Risk Models

In [7]:
model3 = FundamentalFactorRiskModel(factor_covariance=model.factors.cov(),
                                    exposure=model.exposure,
                                    idiosyncratic_risk=model.idiosyncratic_returns.std())

weights = pd.Series(index = prices.columns, data=0.05).values
model3.estimate_risk(weights).value

8.410457095029708e-05

## RiskModel is injected into optimizer

In [8]:
w = cvx.Variable(20, "weights")
w0 = 0.05 * np.ones(20)

objective = cvx.Minimize(model.estimate_risk(w))
constraints = [w >= 0, cvx.sum(w) == 1]

problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(pd.Series(data=w.value, index=prices.columns))
print(model.estimate_risk(w).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

GOOG    2.848336e-18
AAPL    1.119894e-02
FB      2.304546e-18
BABA    3.163122e-02
AMZN    8.130561e-02
GE      2.928897e-02
AMD    -5.601166e-18
WMT     6.158817e-02
BAC    -3.221819e-18
GM     -3.927327e-18
T       9.970227e-02
UAA     3.134359e-18
SHLD    1.459721e-17
XOM     2.298441e-01
RRC    -1.920683e-17
BBY     8.742421e-03
MA      3.316937e-03
PFE     2.484151e-01
JPM    -2.403183e-18
SBUX    1.949663e-01
dtype: float64
4.427599563869687e-05


In [9]:
w = cvx.Variable(20, "weights")
w0 = 0.05 * np.ones(20)

objective = cvx.Minimize(model2.estimate_risk(w))
constraints = [w >= 0, cvx.sum(w) == 1]

problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(pd.Series(data=w.value, index=prices.columns))
print(model.estimate_risk(w).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)


GOOG    1.733346e-18
AAPL    3.460390e-02
FB      4.125759e-18
BABA    2.497240e-02
AMZN    5.801095e-02
GE      2.808894e-02
AMD    -3.642522e-18
WMT     7.364039e-02
BAC    -5.590159e-19
GM     -2.114470e-18
T       1.257405e-01
UAA     3.236164e-18
SHLD    5.489146e-18
XOM     2.209641e-01
RRC    -7.275791e-18
BBY     5.780887e-03
MA      1.931995e-02
PFE     2.300331e-01
JPM    -9.114715e-19
SBUX    1.788449e-01
dtype: float64
4.450529830144715e-05


In [10]:
w = cvx.Variable(20, "weights")
w0 = 0.05 * np.ones(20)

objective = cvx.Minimize(model3.estimate_risk(w))
constraints = [w >= 0, cvx.sum(w) == 1]

problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(pd.Series(data=w.value, index=prices.columns))
print(model.estimate_risk(w).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)


GOOG    2.448818e-07
AAPL    3.714206e-02
FB     -2.935275e-06
BABA    2.076418e-02
AMZN    7.117223e-02
GE      5.390743e-02
AMD     3.182091e-06
WMT     8.615174e-02
BAC     6.963325e-06
GM      2.748607e-06
T       1.295836e-01
UAA     4.368880e-06
SHLD    8.349942e-06
XOM     1.845333e-01
RRC    -3.983659e-06
BBY     1.084189e-02
MA      5.224451e-02
PFE     1.816263e-01
JPM     2.265593e-04
SBUX    1.717872e-01
dtype: float64
4.486586524361176e-05
